In [12]:
import urllib
from urllib.request import urlopen, urlretrieve
from urllib.parse import quote_plus
from bs4 import BeautifulSoup
import regex as re
from tqdm.notebook import tqdm
import os

'''
    ['0', 'Отзывы', 'f=5', 'TOYOTA COROLLA'],
    ['1', 'Двигатель', 'f=8', 'TOYOTA COROLLA'],
    ['2', 'Трансмиссия', '', 'TOYOTA COROLLA'],
    ['3', 'Ходовая', 'f=15', 'TOYOTA COROLLA'],
    ['4', 'Колеса', 'f=11', 'TOYOTA COROLLA'],
    ['5', 'Электрооборудование', 'f=9', 'TOYOTA COROLLA'],
    ['6', 'Мультимедиа', 'f=10', 'TOYOTA COROLLA'],
    ['7', 'Кузов', 'f=7', 'TOYOTA COROLLA'],
    ['8', 'Жидкости', '', 'TOYOTA COROLLA'],
    ['9', 'Сервис', 'f=12', 'TOYOTA COROLLA'],
    ['10', 'Эксплуатация', '', 'TOYOTA COROLLA'],
    ['11', 'Тюнинг', 'f=32', 'TOYOTA COROLLA'],
    ['12', 'Кузов-Вентиляция', 'f=16', 'TOYOTA COROLLA'],
    ['13', 'Сервис-Отзывные компании', 'f=31', 'TOYOTA COROLLA'],
    ['14', 'Тюнинг-Противоугон', 'f=21', 'TOYOTA COROLLA']  
'''

site = 'https://corolla-club.ru/'
folder_names = [
    ['7', 'Кузов', 'f=7', 'TOYOTA COROLLA'],
    ['8', 'Жидкости', '', 'TOYOTA COROLLA'],
    ['9', 'Сервис', 'f=12', 'TOYOTA COROLLA'],
    ['10', 'Эксплуатация', '', 'TOYOTA COROLLA'],
    ['11', 'Тюнинг', 'f=32', 'TOYOTA COROLLA'],
    ['12', 'Кузов-Вентиляция', 'f=16', 'TOYOTA COROLLA'],
    ['13', 'Сервис-Отзывные компании', 'f=31', 'TOYOTA COROLLA'],
    ['14', 'Тюнинг-Противоугон', 'f=21', 'TOYOTA COROLLA']  
]

На странице раздела у нас две задачи:  
1. Найти и сохранить все ссылки на темы
2. Найти ссылку на следующую страницу раздела или убедиться, что это последняя страница раздела.

In [2]:
def collect_themes(site, folder):
    
    site_address = site + 'forumdisplay.php?' + folder
    themes = []
    flag = True

    while flag:
    
        resp = urlopen(site_address) # скачиваем файл
        html = resp.read().decode('utf-8') # считываем содержимое
        soup = BeautifulSoup(html, 'html.parser') # делаем суп
    
        flag = False
        for link in soup.find_all('a'):
            if link.has_attr('href') and link.has_attr('id'):
                if re.search('thread_title_', str(link.get('id'))):
                    s = link.get('href')
                    s = site + 'showthread.php?' + re.search('t=[0-9]+', s)[0]
                    themes.append([link.get_text(), s])
            if not flag and link.has_attr('href') and link.has_attr('rel'):
                if link.get('rel')[0] == 'next':
                    flag = True
                    s = link.get('href')
                    site_address = site + 'forumdisplay.php?' + re.search(folder + '[\S]+', s)[0]
    return themes

На странице темы у нас три задачи:  
1. Понять, что являеся шапкой темы (это сообщение #1), прочитать ее только один раз - НЕ СДЕЛАНО
2. Прочитать и сохранить все сообщения (кроме шапки) на странице темы
3. Найти ссылку на следующую страницу темы или убедиться, что это последняя страница темы.

In [3]:
def collect_messages(site, themes):

    messages = []
    for theme in tqdm(themes):
       
        print(theme[0])
        theme_address = theme[1]
        thread = re.search('t=[0-9]+', theme_address)[0]
        flag = True
        theme_messages = []
        post_data = {}
        
        while flag:
            resp = urlopen(theme_address) # скачиваем файл
            html = resp.read().decode('utf-8', errors='ignore') # считываем содержимое
            soup = BeautifulSoup(html, 'html.parser') # делаем суп
            #Собираем сами сообщения
            lis = soup.find_all('li')
            for li in lis:
                post_id = ''
                post_data = {}
                if li.has_attr('id') and re.search('post_[0-9]+', li['id']):
                    post_id = re.sub('post_', '', li['id'])
                    ##Собираем данные о дате и авторе сообщения
                    tags = li.find_all('span')
                    for span in tags:
                        if span.has_attr('class') and span['class'] == ['date']:
                            post_data[post_id] = [span.get_text().strip()]
                    tags = li.find_all('a')
                    author_id = '0'
                    author_nickname = 'Guest'
                    for tag in tags:
                        if tag.has_attr('class') and tag['class'][0] == 'username':
                            author_id = re.sub('member.php?u=', '', tag['href'])
                            author_nickname = tag.get_text().strip()
                    post_data[post_id].append([author_id, author_nickname])
                    ##Собираем сами сообщения                            
                    tags = li.find_all('div')
                    for div in tags:
                        if div.has_attr('id') and re.search('post_message_', div['id']):
                            blockquote = div.find('blockquote')
                            additional_divs = blockquote.find_all('div')
                            for a_d in additional_divs:
                                a_d.clear()
                            s = blockquote.get_text().strip()
                            s = re.sub('[\']+', '`', s)
                            s = '\'' + re.sub('[\s]+', ' ', s).strip() + '\''
                            if len(s) > 2:
                                theme_messages.append([post_data[post_id][0], post_data[post_id][1][1], s])

            # В отдельном цикле ищем ссылку на следующую страницу темы (это 'a' с атрибутом 'rel', равным 'next')
            # Если ссылка найдена, 
            flag = False 
            for link in soup.find_all('a'):
                if not flag and link.has_attr('href') and link.has_attr('rel'):
                    if link.get('rel')[0] == 'next':
                        flag = True
                        s = link.get('href')
                        theme_address = site + 'showthread.php?' + re.search(thread + '[\S]+', s)[0]
        messages.append(['\''+ theme[0] + '\'', theme_messages])
    return messages

In [4]:
def save_messages(messages, folder_name):
    f_name = os.path.join(folder_name[3], folder_name[3] + '_' + folder_name[0] + '_' + folder_name[1] + '.csv')
    with open(f_name, 'w', encoding='utf-8') as ouf:
        ouf.write('Code,Folder,Theme,Date,Author,Message\n')
        for message in messages:
            header = folder_name[:2] + [message[0]]
            for item in message[1]:
                ouf.write(','.join(header + item))
                ouf.write('\n')

In [5]:
def scraper(site, folder_names):
    for folder_name in folder_names:
        if len(folder_name[2]) > 0:
            folder = folder_name[2]
            current_themes = collect_themes(site, folder)
            current_messages = collect_messages(site, current_themes)
            save_messages(current_messages, folder_name)
    print('Information collected')

In [6]:
%load_ext ipytelegram
%reload_ext ipytelegram
import telepot
bot = telepot.Bot('1301715666:AAGBzLlVDZI7KzGZ_DNyukjauVeTt0QpO-A')
response = bot.getUpdates()
%telegram_setup 1301715666:AAGBzLlVDZI7KzGZ_DNyukjauVeTt0QpO-A 1305740495

In [13]:
%%telegram_send Next message
scraper(site, folder_names)
'information collected'

Аксессуары
Чехлы для сидений
Лобовое стелко - какое и где?
Накладки на пороги.
Кузов оцинкован ?
омыватель камеры заднего вида
Камера переднего обзора
Складывание сидений заднего ряда. Не работает замок
Полез хром на дефлекторах воздуховода
Дефлекторы
Шумоизоляция. Простая, бюджетная.
Клипсы, заглушки
Ограничители дверей.
Сетка на решетку радиатора.
Механическая крутилка направления потока воздуха. Дилер не помог.
Какой объём топливного бака
Локеры (защитные подкрылки)
болт крепления задней спинки сиденья.
как подчинить ручку в салоне
номер оригинальной магнитолы
Чем заряжаете телефон?
Накладка крыла перед зеркалом
коврики в салон
Обивка сидений е160
Сырость в багажнике. Неприятный запах тухлятины
Скрипит дверь
Нужен совет по подлокотнику в Е 150 2007 г.в.
Замена ткани на подлокотнике и дверных ручек
Автобокс
СЛОМАЛСЯ ПЕРЧАТОЧНЫЙ ЯЩИК
Как снять форсунку омывателя стекла??
крышка багажника новой короллы
как снять накладку ручки открывания двери внутри салона?
Подсветка бардачка
Амортиза

Пищание открытой водительской двери
Транспортировка велосипедов. Велокрепления
Водительская дверь
Коврик в багажник
Конденсат в багажнике под запаской
свист на скорости
Подсветка заднего номера
уплотнители передних дверей
Помутнение ЛКП после 7000 км бережной эксплуатации.
Дефект окраски кузова?
Запах в салоне
Сечатые шторки на авто
Щебень в лобовое кто как боролся??
Задний диван на 121 кузов
Решетка радиатора
Свистит вентилятор печки на 1 и 2 скорости
Аудиосистема Toyota Touch второго поколения, что может воспроизводить?
подкрылки
Как сложить зеркала? help ;)
посторонний звук пластмассы около левого уха
Скрежет обшивки задних арок.
Задние противотуманные фары
Утеплитель переднего капота.
Замок багажника
треск сзади при минусовой температуре.
Органайзер
Подскажите, что это?  :)
Подогрев задних сидений.
Элементы оформления интерьера от последних комплектаций
Крепления грузов в багажнике
Грязь в стекла-разочарование
Лобовое стекло
Решетка радиатора
открытие капота
Задиры резинового уплот

ТО 3
ТО 5
Регламент технического обслуживания (карта ТО)
ТО 4
ТО 2
Каталог запчастей на новую Toyota Corolla (160-180)
Гарантия на Corolla
Нулевое ТО (бесплатное) - 1000км
ТО 1
Кузовной ремонт е150 выбор запчастей
Повышенные обороты на P
Свечи зажигания e160-180
Уровень антифриза на e160
Ремкомплект рулевой рейки (ЭУР)
Удаление Катализатора.
Задний тормозной суппорт
Ступица передняя или ступичный подшипник
Тормозные колодки
Кто менял топливный фильтр?
Вопрос по маслу 0W-20
Лучшие фирмы заменители
Стойки стабилизатора передние какие выбрать ?
Как снять Штатное ГУ Е180
Corolla e 150
Выбор магнитолы.
Запах бензина на ходу при открытых окнах.
Не поворачивается замок зажигания.
Запчасти от e150 которые подходят е180
Сняли передок помогите собрать заново
Аналоги запчастей на Toyota Corolla 160
ЦЕПЬ ГРМ
Вариатор на спуске.
кузовной ремонт в Москве
Какое моторное масло самое оптимальное?
Дроссельная заслонка- очистка и адаптация.
Подмена запасных частей на сервисе
Печка дует холодным пожалуйст

Ремонт двери. Прошу совета.
Кожух на рулевой колонке!
Взодушный фильтр 17801-21050 (япония) и 17801-0D060 (польша)
TOYOTA COROLLA EE103   5Е-FE 2000 год
Сервис по-пермски - бессмысленно и беспощадно.
Украли "Накладка зеркала" / "Корпус зеркала"
Рвзмеры пыльника ШРУСа наружнего
сбрасывает передачу в нейтраль
Компрессор в машину
Как устранить прокур на напольном покрытии???
Пробоина в топливном баке
пикает зуммер при движении
КАСКО 2 год
Сгорели мозги??
Подъедает резину на переднем левом колесе
Штатный задний подкрылок
Влетел в яму... подскажите!
Цена авто?
стук в подвеске справа впереди
Цокание в двигателе 1ZRFE 1.6
Еще один бедолага попал на капиталку
Отказ в гарантийном ремонте двигателя
обрыв троса мкпп
Кто знает что за болт на 1zr-fe сзади?
Обкатка
Ремонт катафота в заднем бампере
Toyota Corolla 2003 D4D 2.0 1CD-FTV не набирает обороты
подключение прикуривателя для задних пассажиров
Двери
Синтетические масла и все что с ними связано.
Нужна электрическая схема разъёма управления магн

Замена втулок стабилизатора на Королле 160
Избавляемся от неприятного запаха их кондиционера/печки
Установка ходовых огней.
Дооснащение Corolla 180 штатной системой контроля давления в шинах — TPMS.
Очечник
Установка твитеров на Corolla 2013-2016
КОЛХОЗ КОРОЛЛЫ ЗАКОНЧИЛ !!! (почти)  ;))))))
Самостоятельное подключение радардетектора - видеорегистратора
Установка демпфера руля (гасителя вибраций)
Провернулась гайка в подрамнике
Писюн регулируемый?!
Новый ключ с кнопками E160-180 дорестайл
Самооткрывающийся багажник
Установка линзованных ПТФ
Бардачок в карпет
кнопки на руле без адаптера
Чем склеить пластик?!
Установка оригинальной линзированной оптики на e160-180
Крюк в багажнике
Салонное зеркало заднего вида
Установка камеры заднего вида на короллу 150,151
Обшиваем руль эко кожей.
Замена простой приборки на приборку от максималки.
Зимняя резина
Светодиодная лента на двери
Утепление E160
Не закрывается багажник в мороз
Устойчивость антикорра кузова к самодельщине?
ошибка p0560
Подключени

Работа кондиционера
Заклинило "крутилку" регулировки температуры (комплектация с кондиционером)
Не включается кондиционер
Климат-контроль
Крутилка регулировки потоков воздуха
Чистка испарителя кондиционера
Справляется ли кондиционер/климат-контроль с охлаждением авто.
воздушный фильтр салона и двигателя
Конденсат
ЗАМЕРЗЛИ НОГИ..
положение заслонки направления потока воздуха зимой
печка и фары не работает одновременно
Принцип работы "двухзонного" климата
Салонный фильтр
Запотевание стекол !!!
Подогреватель РТС
Как достать детальку LEGO из системы обдува ?)
Не работает печка
Присутствует ли у рестайлинговой модели фен для обогрева салона
Кондиционер. Как пользоваться
Печь дают не такой теплый воздух как раньше!
Раскачка при включенном кондее
из воздуховодов летит белая пыль
Температура за бортом
Свист при включенном отопителе
Хруст заслонки регулирующей температуру
Поддувает холодный воздух в районе панели приборов.
Дефлекторы сзади
Запах сырости при включении климат-контроля...
что за к

[TSB] Неустойчивая работа двигателя во время разгона из-за неисправности блока клапанов
[TSB] Двигатель NR: расход масла
[TSB] Двигатели серии ZR: затрудненный запуск из-за загрязнения форсунок
[TSB] Ударный шум со стороны ограничителя открывания задней двери
[TSB] Визг задних тормозов при торможении на небольшой скорости
[TSB] Визг сцепления при трогании автомобиля с места (двигатели 1ZR и 2ZR)
[TSB] Двигатели серии ZR: подвреждения цилиндра выключения сцепления в сборе с подшипником (подшипника CSC)
[TSB] Стучащий шум со стороны втулок штанги переднего стабилизатора поперечной устойчивости.
[TSB] Неестественный шум трения / царапания, доносящийся из задней части автомобиля во время поворота
[TSB] EC6#: воющий шум при движении на всех передачах (без повреждения)
[TSB] Вибрация при нажатии/отпускании педали сцепления.
[TSB] Замена ветрового стекла на автомобилях с нагревательным элементом для датчика системыTSS
[TSB] Отображение предупреждающего сообщения TSS из-заблокировки обзора
[M]

Карты установки сигнализаций на Corolla e160-e180
Непонятное устройство
CAN-шина под капотом
Механические противоугонные системы
Принцип работы штатной сигнализации
Что блокировать сигналкой?
Угнали ласточку
Выбор сигнализации
Самостоятельная установка сигнализации StarLine A94 CAN на короллу 160
Бесключевой обходчик иммо теперь доступен у старлайна
Кто ставил сигналку с автозапуском?
Вопрос владельцам сигнализации Старлайн А93
Вопрос по сигнализации.
Выбор замка капота
Защитный бокс на ECU
Функция энергосбережения.
Попытка угона.
При наличиии КАСКО будете ли Вы вкладываться в противоугонные системы?
Закрытие замка дверей при начале движения
Предлагаю поделиться опытом использования дополнительного иммобилайзейра
Автономный маяк GPS
Самостоятельная установка доп. иммобилайзера
штатный радиоканал
Pandora замок капота
Угнали :(
DLC3
Установка сигнализации у ОД

Information collected


'information collected'

In [ ]:
import pandas as pd
mess = pd.read_csv('TOYOTA COROLLA_11_Тюнинг.csv', quotechar="'")
mess

In [6]:
for folder_name in folder_names:
    if len(folder_name[2]) > 0:
        folder = folder_name[2]
        current_themes = collect_themes(site, folder)

In [ ]:
current_themes

In [ ]:
scraper(site, folder_names)